## MCM Problem

In [4]:
'''
int n: the number of matrices + 1
list d: sizes of matrices with (n+1) elements
'''
import numpy as np



def MCM(d, n):
	# add more rows/columns to facilitate the indexing
	M = np.zeros(shape=((n+1, n+1)), dtype=int)
	K = np.zeros(shape=((n+1, n+1)), dtype=int)  # accumulating index of parentheses
	for i in range(n):
		M[i, i] = 0  # base case
	for s in range(2, n):  # go diagonally
		for i in range(1, n-s+1):
			min = 1e5
			j = i + s - 1
			for k in range(i, j):
				temp = M[i, k] + M[k+1, j] + d[i-1]*d[k]*d[j]
				if temp < min:
					min = temp
					K[i, j] = k
			M[i, j] = min
	# refining the final results
	M = np.delete(M, [0, M.shape[0]-1], 0)
	M = np.delete(M, [0, M.shape[1]-1], 1)
	K = np.delete(K, [0, K.shape[0]-1], 0)
	K = np.delete(K, [0, K.shape[1]-1], 1)

	return M, K


def find_sol():
	M, K = MCM(d, n)
	i, j = 0, n-2
	cur = K[i, j]
	index = []
	stop = False  # stop when the size of all problems do not exceed 1

	while not stop:
		cur = K[i, j]
		index.append(cur)
		par = [[i, cur-1], [cur, j]]  # two subproblems
		i = cur
		if par[0][1] - par[0][0] <= 1 and par[1][1] - par[1][0] <= 1:
			stop=True
	
	return index


def print_sol():
	M, K = MCM(d, n)
	table = [[f'{M[i, j]}/{K[i, j]}' for j in range(n-1)] for i in range(n-1)]
	for row in table:
		for ele in row:
			print(ele.rjust(10), end=' ')
		print()
	print('Minimum number of scalar multiplcations needed:', M[0, n-2])

In [5]:
if __name__ == '__main__':
	print('DYNAMIC PROGRAMMING: Matrix Chain Multiplication')
	print()

	## TYPE YOUR INPUT HERE ================== ##
	d = [10, 100, 20, 5]  # list of sizes
	## ======================================= ##
	n = len(d)  # number of matrix + 1
	## ======================================= ##
	print('Sizes list:', d)
	print('The number of matrices:', n-1)
	print('===== RESULT =====')
	print()
	print_sol()
	#print('Index of parentheses (index of matrix starts at 1):', find_sol())

DYNAMIC PROGRAMMING: Matrix Chain Multiplication

Sizes list: [10, 100, 20, 5]
The number of matrices: 3
===== RESULT =====

       0/0    20000/1    15000/1 
       0/0        0/0    10000/2 
       0/0        0/0        0/0 
Minimum number of scalar multiplcations needed: 15000


## Change making problem

In [4]:
'''
N: the amount to return
list d: value of each coin 
n: number of denominations
'''
import numpy as np


def making_change(d, n, N):
	t = np.zeros(shape=(n, N+1), dtype=int)
	for i in range(n):
		t[i, 0] = 0  # base case

	for i in range(n):
		for j in range(N+1):
			if i == 0:
				t[i, j] = t[i, j-d[i]] + 1
			elif j < d[i]:
				t[i, j] = t[i-1, j] 
			else:
				t[i, j] = min(t[i-1, j], t[i, j-d[i]] + 1)

	for i in range(n):  # all element - 1 due to the index in Python
		for j in range(N+1):
			t[i][j] -= 1
	return t


def find_sol():
	t = making_change(d, n, N)
	i, j = n-1, N 
	cur = t[n-1, N]
	used = []
	stop = False
	while not stop:
		cur = t[i, j]
		if cur == t[i-1, j]:
			i, j = i-1, j 
		if cur == (t[i, j-d[i]] + 1):
			used.append(d[i])
			i, j = i, j-d[i]
		if sum(used) == N:
			stop=True

	return used



def print_sol():
	table = making_change(d, n, N)
	print('Coins/Amounts'.ljust(15), end='| ')
	for i in range(N):
		print(f'{i:3}', end=' | ')
	print(f'{N:3} | ')
	print('-' * (16+6*(N+1)))
	for i in range(len(table)):
		print(f'd[{i+1}] = {d[i]}'.ljust(15), end='| ')
		for j in table[i]:
			print(f'{j:3}', end=' | ')
		print()
	print()
	print('Minimum number of coins needed:', table[n-1, N])

In [5]:
if __name__ == '__main__':
	print('DYNAMIC PROGRAMMING: Change-making problem')
	print()

	## TYPE YOUR INPUT HERE ================== ##
	d = [1, 2, 3]  # list of value of each denomination
	N = 7  # the money we need to return
	## ======================================= ##
	n = len(d)
	## ======================================= ##
	print('The number of denominations:', n)
	print('Value of each denomination:', d)
	print('The amount of money we need to return:', N)
	print('===== RESULT =====')
	print()
	print_sol()
	print('Coins used:', find_sol())



DYNAMIC PROGRAMMING: Change-making problem

The number of denominations: 3
Value of each denomination: [1, 2, 3]
The amount of money we need to return: 7
===== RESULT =====

Coins/Amounts  |   0 |   1 |   2 |   3 |   4 |   5 |   6 |   7 | 
----------------------------------------------------------------
d[1] = 1       |   0 |   1 |   2 |   3 |   4 |   5 |   6 |   7 | 
d[2] = 2       |   0 |   1 |   1 |   2 |   2 |   3 |   3 |   4 | 
d[3] = 3       |   0 |   1 |   1 |   1 |   2 |   2 |   2 |   3 | 

Minimum number of coins needed: 3
Coins used: [3, 2, 2]


## Floyd-Warshall Algorithm

In [6]:
import numpy as np


def floyd(L):
	n = len(L) .
	D = L
	P = np.zeros(shape=(n, n), dtype=int)  # accumulating the index

	for k in range(n):
		for i in range(n):
			for j in range(n):
				temp = D[i, k] + D[k, j]
				if temp < D[i, j]:
					D[i, j] = temp
					P[i, j] = k+1

	return D, P


def print_sol(L):  # just like the main function, but will not return D and P
	n = len(L)
	D = L
	P = np.zeros(shape=(n, n), dtype=int)  # accumulating the index

	for k in range(n):
		for i in range(n):
			for j in range(n):
				temp = D[i, k] + D[k, j]
				if temp < D[i, j]:
					D[i, j] = temp
					P[i, j] = k+1
		print(f'D{k+1} = {D}')  # I tried using list to store the intermediary results, but did not work???
		print(f'P{k+1} = {P}')
		print()


def find_sol(D, P, i, j):
	# D, P = floyd(L)
	# print(D)
	# print(P)
	path = [i]
	i, j = i-1, j-1
	stop = False
	
	while not stop:
		node = P[i, j]
		if node != 0:
			path.append(node)
		i = node-1

		if node == 0:
			stop = True

	path.append(j+1)
	return path

In [7]:
if __name__ == '__main__':
	print('DYNAMIC PROGRAMMING: Floyd-Warshall Algorithm')
	print()
	INF = 99999

	## TYPE YOUR INPUT HERE ================== ##
	# distance matrix, use numpy
	L = np.array([
		[0, 3, 8, INF, 4],
		[INF, 0, INF, 1, 7],
		[INF, 4, 0, INF, INF],
		[2, INF, 5, 0, INF],
		[INF, INF, INF, 6, 0]
		])  
	## ======================================= ##
	print('===== RESULT =====')
	print()
	print('Process:')
	print_sol(L)

	# NOTE: Uncomment the following codes if you want to find the detailed path
	# D, P = floyd(L)
	# start = 5  # index starts at 1
	# goal = 2
	# print(f'Path from {start} to {goal}: {find_sol(D, P, start, goal)}')


DYNAMIC PROGRAMMING: Floyd-Warshall Algorithm

===== RESULT =====

Process:
D1 = [[    0     3     8 99999     4]
 [99999     0 99999     1     7]
 [99999     4     0 99999 99999]
 [    2     5     5     0     6]
 [99999 99999 99999     6     0]]
P1 = [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 1 0 0 1]
 [0 0 0 0 0]]

D2 = [[    0     3     8     4     4]
 [99999     0 99999     1     7]
 [99999     4     0     5    11]
 [    2     5     5     0     6]
 [99999 99999 99999     6     0]]
P2 = [[0 0 0 2 0]
 [0 0 0 0 0]
 [0 0 0 2 2]
 [0 1 0 0 1]
 [0 0 0 0 0]]

D3 = [[    0     3     8     4     4]
 [99999     0 99999     1     7]
 [99999     4     0     5    11]
 [    2     5     5     0     6]
 [99999 99999 99999     6     0]]
P3 = [[0 0 0 2 0]
 [0 0 0 0 0]
 [0 0 0 2 2]
 [0 1 0 0 1]
 [0 0 0 0 0]]

D4 = [[ 0  3  8  4  4]
 [ 3  0  6  1  7]
 [ 7  4  0  5 11]
 [ 2  5  5  0  6]
 [ 8 11 11  6  0]]
P4 = [[0 0 0 2 0]
 [4 0 4 0 0]
 [4 0 0 2 2]
 [0 1 0 0 1]
 [4 4 4 0 0]]

D5 = [[ 0  3  8  4  4]
 [ 3

## 0-1 Knapsack Problem

In [8]:
'''
list w: weight of each item 
list v: value of each item 
n: total number of items
W: initial capacity of the knapsack
'''
import numpy as np


def knapsack(W, w, v, n):
	K = np.zeros(shape=((n, W+1)), dtype=int)

	for i in range(n):
		K[i, 0] = 0  # base case
	for j in range(W+1):
		if w[0] <= j:
			K[0, j] = v[0]
		else:
			K[0, j] = 0 
	for i in range(1, n):
		for j in range(W+1):
			if j < w[i]:
				K[i, j] = K[i-1, j]
			else:
				K[i, j] = max(K[i-1, j], K[i-1, j-w[i]]+v[i])
	return K


def find_sol():
	t = knapsack(W, w, v, n)
	i, j = n-1, W 
	cur = t[i, j]
	used = []
	stop = False
	sum_weight = []

	while not stop:
		cur = t[i, j]
		if cur == t[i-1, j]:
			i, j = i-1, j 
		else:
			used.append(i+1)
			sum_weight.append(w[i])
			i, j = i-1, j-w[i]
		if sum(sum_weight) == W:
			stop = True
			
	return used


def print_sol():
	table = knapsack(W, w, v, n)
	print('i/j'.ljust(10), end='| ')
	for i in range(W):
		print(f'{i:3}', end=' | ')
	print(f'{W:3} | ')
	print('-' * (11+6*(W+1)))
	for i in range(len(table)):
		print(f'w[{i+1}] = {w[i]}'.ljust(10), end='| ')
		for j in table[i]:
			print(f'{j:3}', end=' | ')
		print()
	print()
	print('Maximum value in the knapsack:', table[n-1, W])

In [9]:
if __name__ == '__main__':
	print('DYNAMIC PROGRAMMING: 0-1 Knapsack problem')
	print()

	## TYPE YOUR INPUT HERE ================== ##
	w = [10, 20, 30]  # list of weights
	v = [60, 100, 120] # list of value
	W = 50  # initial capacity
	## ======================================= ##
	n = len(w)  # number of items
	## ======================================= ##
	print('Weight list:', w)
	print('Value list:', v)
	print('The number of items:', n)
	print('Initial capacity:', W)
	print('===== RESULT =====')
	print()
	print_sol()
	print('Items in the knapsack:', find_sol())


DYNAMIC PROGRAMMING: 0-1 Knapsack problem

Weight list: [10, 20, 30]
Value list: [60, 100, 120]
The number of items: 3
Initial capacity: 50
===== RESULT =====

i/j       |   0 |   1 |   2 |   3 |   4 |   5 |   6 |   7 |   8 |   9 |  10 |  11 |  12 |  13 |  14 |  15 |  16 |  17 |  18 |  19 |  20 |  21 |  22 |  23 |  24 |  25 |  26 |  27 |  28 |  29 |  30 |  31 |  32 |  33 |  34 |  35 |  36 |  37 |  38 |  39 |  40 |  41 |  42 |  43 |  44 |  45 |  46 |  47 |  48 |  49 |  50 | 
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
w[1] = 10 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |  60 |  60 |  60 |  60 |  60 |  60 |  60 |  60 |  60 |  60 |  60 |  60 |  60 |  60 |  60 |  60 |  60 |  60 |  60 |  60 |  60 |  60 |